# Run Functions to Add Information to Projects

In [1]:
# ! pip install nltk

In [1]:
import numpy as np
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

from shared_utils import geography_utils

import dla_utils

import utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

## Read in Data and function development

In [3]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [4]:
proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_(IIJA_Reporting)_ (3-9-23).xlsx", header=[3], skiprows=range(4,1784)))

In [5]:
proj.drop(columns =['unnamed:_0'], axis=1, inplace=True)

In [6]:
proj['summary_recipient_defined_text_field_1_value'] = proj['summary_recipient_defined_text_field_1_value'].fillna(value='None')


In [7]:
new_codes = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FY21-22ProgramCodesAsOf5-25-2022.v2.xlsx"))
code_map = dict(new_codes[['iija_program_code', 'new_description']].values)
proj['program_code_description'] = proj.program_code.map(code_map)

In [8]:
proj.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value
6157,2022-09-13,ZS3E,NaN,5006843,0716000354L,"IN THE CITY OF LOS ANGELES, CITYWIDE LIST OF 28 LOCATIONS, STARTING WITH ADAMS BL & WEST VIEW ST, AVALON BL & 90TH ST, AVALOS BL & 97TH ST, ETC. INST",37,Cong Dist 40,Active,"IN THE CITY OF LOS ANGELES, CITYWIDE LIST OF 28 LOCATIONS, STARTING WITH ADAMS BL & WEST VIEW ST, AVALON BL & 90TH ST, AVALOS BL & 97TH ST, ETC. INSTALL RECTANGULAR RAPID FLASHING BEACONS (RRFBS) AT UNCONTROLLED LOCATIONS",21,Safety,715042.35,285179.4,L5006SCAG


In [9]:

# df = utils.read_data_all()
    
    ## function that adds known agency name to df 
df = utils.identify_agency(proj, 'summary_recipient_defined_text_field_1_value')

In [10]:
df.sample(3)

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
6304,2023-02-22,LY40,NaN,5109245,0618000219L,"SR99/ SR58 CONNECTOR, SR99 FROM BELLE TERRACE TO MING AVE. IMPROVEMENTS ON SR99/ SR58 CONNECTOR INCLUDING THE CONSTRUCTION OF A COLLECTOR-DISTRIBUTOR",29,Cong Dist 23,Active,"SR99/ SR58 CONNECTOR, SR99 FROM BELLE TERRACE TO MING AVE. IMPROVEMENTS ON SR99/ SR58 CONNECTOR INCLUDING THE CONSTRUCTION OF A COLLECTOR-DISTRIBUTOR AND AUXILIARY LANE ON SR99.",44,Other,53133355.00,11121638.38,L5109KCOG,5109.0,Bakersfield,6.0,Kern County,Kern County Council of Governments,Kern County Council Of Governments
1946,2022-05-31,L1C0,NaN,5006195,07931669L,"LAUREL CANYON OVER TUJUNGA WASH, BRIDGE # 53C1233 BRIDGE REHABILITATION",37,Cong Dist 29,Closed,"LAUREL CANYON OVER TUJUNGA WASH, BRIDGE # 53C1233 BRIDGE REHABILITATION",16,Right of Way,126300.00,111813.00,L5006SCAG,5006.0,Los Angeles,7.0,Los Angeles County,Los Angeles County Metropolitan Transportation Auth.,Southern California Association Of Governments
10714,2023-03-07,Z240,NaN,000C477,0213000032S,"IN BUTTE, LASSEN, PLUMAS, SHASTA, SISKIYOU, AND TRINITY COUNTIES AT VARIOUS LOCATIONS. IMPROVE CHAIN CONTROL AREAS (TC)",105,Cong Dist 2,Active,"IN BUTTE, LASSEN, PLUMAS, SHASTA, SISKIYOU, AND TRINITY COUNTIES AT VARIOUS LOCATIONS. IMPROVE CHAIN CONTROL AREAS (TC)",17,Construction Engineering,61585.19,61585.19,S SHASTA,NaN,California,2.0,Trinity County,NaN,NaN


In [11]:
len(df>>filter(_.implementing_agency=='Unknown'))

7

In [12]:
len(df>>filter(_.county_name=='Unknown'))

67

## Test & Export

In [14]:
# GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [15]:
df.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
2424,2022-05-05,YS32,Section 164 Penalties - Use for HSIP Activities,P135009,0517000184S,"SANTA BARBARA COUNTY IN SANTA MARIA AT BUNNY AVENUE AND FROM ROEMER WAY TO PREISKER LANE. INSTALL SIGNAL, LIGHTING, CURB RAMPS AND SIDEWALK.",83,Cong Dist 24,Active,"ON STATE ROUTE: 135. SANTA BARBARA COUNTY IN SANTA MARIA AT BUNNY AVENUE AND FROM ROEMER WAY TO PREISKER LANE. INSTALL SIGNAL, LIGHTING, CURB RAMPS AND SIDEWALK.",17,Construction Engineering,570000.0,513000.0,S SBCAG,NaN,California,5.0,Santa Barbara County,NaN,NaN


In [14]:
###test agg. PASS df with title column, concat and 552 rows
##throwingerror
agg = utils.get_clean_data(df, full_or_agg = 'agg')

In [15]:
sorted(list(agg.project_title_new))

[' Added Roadway Capacity in Caltrans',
 ' Added Roadway Capacity in Caltrans',
 ' Added Roadway Capacity in Industry',
 ' Added Roadway Capacity in Kern County',
 ' Added Roadway Capacity in Los Angeles',
 ' Added Roadway Capacity in Los Angeles',
 ' Added Roadway Capacity in Los Angeles County',
 ' Added Roadway Capacity in Los Angeles County & San Mateo County',
 ' Added Roadway Capacity in Madera County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Murrieta',
 ' Added Roadway Capacity in Needles',
 ' Added Roadway Capacity in Orange County',
 ' Added Roadway Capacity in Orange County',
 ' Added Roadway Capacity in Riverside County',
 ' Added Roadway Capacity in Riverside County',
 ' Added Roadway Capacity in San Bernardino County',
 ' Added Roadway Capacity in San Bernardino County',
 ' Added Roadway Capacity in San Bernardino County',
 ' Added Roadway Capacity in Santa Clara Valley Transportation Authority',
 ' Added Roadway Capa

In [16]:
len(agg)

3155

In [17]:
agg = utils.title_column_names(agg)

In [25]:
# agg.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_3-9-23_agg.csv")

In [19]:
###test full. PASS title, no concat and 1241 rows
full = utils.get_clean_data(df, full_or_agg = 'full')

In [20]:
full = utils.title_column_names(full)

In [27]:
# full.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_3-9-23_all.csv")

## Check Titles

In [24]:
agg = to_snakecase(agg)

In [44]:
test_df = agg>>filter(_.project_title_new.str.contains('Unknown')
                      #| _.project_title_new.str.contains("Statewide")
                     )

In [45]:
test_df.sample()

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,county_name_title,implementing_agency_locode,rtpa_name,mpo_name,project_title_new
1477,2022-08-04,130R295,Unknown,L7510SCAG,MS50 | ZS5E,nan,0823000009L,15,Preliminary Engineering,"RADIO ROAD GRADE CROSSING AT THE BNSF RAILROAD XING, IN THE CITY OF CORONA SECTION 130 RAILROAD-HIGHWAY GRADE CROSSING HAZARDS ELIMINATION",1760000.0,Cong Dist 42,nan,65,Unknown,Unknown,7510.0,None,None,Preliminary Engineering Projects in Unknown


In [46]:
locodes = to_snakecase(pd.read_excel(f"gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/locodes_updated7122021.xlsx"))

In [47]:
locodes['agency_name'] = locodes['agency_name'].str.upper()

In [48]:
locode_names = list(locodes['agency_name'].unique())

In [49]:
locode_names.remove('ROSS')


In [50]:
# locode_names

In [51]:
pattern = '|'.join(locode_names)

In [52]:
test_df.sample()

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,county_name_title,implementing_agency_locode,rtpa_name,mpo_name,project_title_new
2195,2022-11-07,6249015,Unknown,02242009L6249,H400 | L400 | Q400,nan,04924225L,15,Preliminary Engineering,"EASTERN SOLANO COUNTY , SPARE THE AIR PROGRAM FY 03/04 .",769999.53,Cong Dist 3,nan,95,Unknown,Unknown,2242.0,None,None,Preliminary Engineering Projects in Unknown


In [53]:
test_df.project_title.str.contains(pattern)

/tmp/ipykernel_2067/2618956443.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


1451    True
1477    True
2195    True
Name: project_title, dtype: bool

In [54]:
### https://stackoverflow.com/questions/68869434/create-an-pandas-column-if-a-string-from-a-list-matches-from-another-column
#test_df["agencies_found"] = test_df["project_title"].apply(lambda x: next((a for a in locode_names if a in x), ""))
test_df["agencies_found2"] = test_df["project_title"].map(lambda s: next((name for name in locode_names if name in s), np.nan))

/tmp/ipykernel_2067/35383681.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [55]:
test_df["agencies_found2"] = test_df["agencies_found2"].str.title()

/tmp/ipykernel_2067/2687676984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [56]:
test_df

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,county_name_title,implementing_agency_locode,rtpa_name,mpo_name,project_title_new,agencies_found2
1451,2022-08-02,130R293,Unknown,L7510FCOG,LS40 | ZS5E,nan,0623000018L,15,Preliminary Engineering,"SHIELDS AVE GRADE CROSSING AT THE BNSF RR XING, IN THE CITY OF FRESNO SECTION 130 RAILROAD-HIGHWAY GRADE CROSSING HAZARDS ELIMINATION",2968000.00,Cong Dist 16,nan,19,Unknown,Unknown,7510.0,None,None,Preliminary Engineering Projects in Unknown,Fresno
1477,2022-08-04,130R295,Unknown,L7510SCAG,MS50 | ZS5E,nan,0823000009L,15,Preliminary Engineering,"RADIO ROAD GRADE CROSSING AT THE BNSF RAILROAD XING, IN THE CITY OF CORONA SECTION 130 RAILROAD-HIGHWAY GRADE CROSSING HAZARDS ELIMINATION",1760000.00,Cong Dist 42,nan,65,Unknown,Unknown,7510.0,None,None,Preliminary Engineering Projects in Unknown,Corona
2195,2022-11-07,6249015,Unknown,02242009L6249,H400 | L400 | Q400,nan,04924225L,15,Preliminary Engineering,"EASTERN SOLANO COUNTY , SPARE THE AIR PROGRAM FY 03/04 .",769999.53,Cong Dist 3,nan,95,Unknown,Unknown,2242.0,None,None,Preliminary Engineering Projects in Unknown,Solano County


In [22]:
agg>>filter(_["Project Title New"].str.contains('Unknown'))

,Fmis Transaction Date,Project Number,Implementing Agency,Summary Recipient Defined Text Field 1 Value,Program Code,Program Code Description,Recipient Project Number,Improvement Type,Improvement Type Description,Project Title,Obligations Amount,Congressional District,District,County Code,County Name,County Name Title,Implementing Agency Locode,Rtpa Name,Mpo Name,Project Title New


In [23]:
agg>>filter(_["Implementing Agency"].str.contains('Unknown'))

,Fmis Transaction Date,Project Number,Implementing Agency,Summary Recipient Defined Text Field 1 Value,Program Code,Program Code Description,Recipient Project Number,Improvement Type,Improvement Type Description,Project Title,Obligations Amount,Congressional District,District,County Code,County Name,County Name Title,Implementing Agency Locode,Rtpa Name,Mpo Name,Project Title New
1451,2022-08-02,130R293,Unknown,L7510FCOG,LS40 | ZS5E,nan,0623000018L,15,Preliminary Engineering,"SHIELDS AVE GRADE CROSSING AT THE BNSF RR XING, IN THE CITY OF FRESNO SECTION 130 RAILROAD-HIGHWAY GRADE CROSSING HAZARDS ELIMINATION",2968000.00,Cong Dist 16,nan,19,Unknown,Unknown,7510.0,None,None,Preliminary Engineering Projects in Fresno
1477,2022-08-04,130R295,Unknown,L7510SCAG,MS50 | ZS5E,nan,0823000009L,15,Preliminary Engineering,"RADIO ROAD GRADE CROSSING AT THE BNSF RAILROAD XING, IN THE CITY OF CORONA SECTION 130 RAILROAD-HIGHWAY GRADE CROSSING HAZARDS ELIMINATION",1760000.00,Cong Dist 42,nan,65,Unknown,Unknown,7510.0,None,None,Preliminary Engineering Projects in Corona
2195,2022-11-07,6249015,Unknown,02242009L6249,H400 | L400 | Q400,nan,04924225L,15,Preliminary Engineering,"EASTERN SOLANO COUNTY , SPARE THE AIR PROGRAM FY 03/04 .",769999.53,Cong Dist 3,nan,95,Unknown,Unknown,2242.0,None,None,Preliminary Engineering Projects in Solano County
